In [13]:
#Imports
import nltk
import os
import tensorflow as tf
import json
import pickle
from collections import Counter
#Loading Data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import Model,Input
import joblib

In [2]:
with open("intents.json") as file:
    data = json.load(file)

In [3]:
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

#Looping through our data
for intent in data['intents']:
    for pattern in intent['patterns']:
        pattern = pattern.lower()
        docs_x.append(pattern)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [4]:
len(labels)

10

In [5]:
docs_x

['hi',
 'hey',
 'how are you',
 'is anyone there ?',
 'hello',
 'good day',
 'whats up',
 'book a table',
 'can i book a table ?',
 'i want to book a table',
 'book seat',
 'i want to book a seat',
 'can i book a seat ?',
 'how many seats are available ?',
 'available seats',
 'how many tables are available ?',
 'available tables',
 'cya',
 'see you later',
 'goodbye',
 'i am leaving',
 'have a good day',
 'cya later',
 'i gotta go now',
 'i gotta rush now',
 'how old',
 'how old is dexter',
 'what is your age',
 'how old are you',
 'age ?',
 'what is your name',
 'what should i call you',
 'whats your name ?',
 'when are you guys open',
 'what are your hours',
 'hours of operation',
 'hours',
 'what is the timing',
 'id like to order something',
 'whats on the menu',
 'what do you reccommend ?',
 'could i get something to eat',
 'im hungry',
 'contact information',
 'contact us',
 'how can i contact you',
 'can i get the contact details',
 'i wanna give some feedback',
 'how can i giv

In [6]:
my_counter = Counter()

for sentence in docs_x:
    my_counter.update(sentence.split())

In [7]:
count_dict = dict(my_counter)

In [8]:
count_dict

{'hi': 1,
 'hey': 1,
 'how': 8,
 'are': 7,
 'you': 8,
 'is': 7,
 'anyone': 1,
 'there': 1,
 '?': 13,
 'hello': 1,
 'good': 2,
 'day': 2,
 'whats': 5,
 'up': 1,
 'book': 6,
 'a': 6,
 'table': 3,
 'can': 5,
 'i': 13,
 'want': 2,
 'to': 4,
 'seat': 3,
 'many': 2,
 'seats': 2,
 'available': 4,
 'tables': 2,
 'cya': 2,
 'see': 1,
 'later': 2,
 'goodbye': 1,
 'am': 1,
 'leaving': 1,
 'have': 1,
 'gotta': 2,
 'go': 1,
 'now': 2,
 'rush': 1,
 'old': 3,
 'dexter': 1,
 'what': 7,
 'your': 4,
 'age': 2,
 'name': 2,
 'should': 1,
 'call': 1,
 'when': 1,
 'guys': 1,
 'open': 1,
 'hours': 3,
 'of': 1,
 'operation': 1,
 'the': 7,
 'timing': 1,
 'id': 1,
 'like': 1,
 'order': 1,
 'something': 2,
 'on': 1,
 'menu': 1,
 'do': 1,
 'reccommend': 1,
 'could': 1,
 'get': 2,
 'eat': 1,
 'im': 1,
 'hungry': 1,
 'contact': 4,
 'information': 1,
 'us': 1,
 'details': 1,
 'wanna': 1,
 'give': 2,
 'some': 2,
 'feedback': 2,
 'location': 2,
 'where': 2,
 'locatated': 1,
 'restaurant': 1,
 'located': 1,
 'address':

In [9]:
sorted_count_dict = dict(sorted(count_dict.items(), key=lambda kv: kv[1]))

In [10]:
vectorizer = CountVectorizer(vocabulary=list(sorted_count_dict.keys()), lowercase=False, binary=True)

In [11]:
vectorizer.fit(docs_x)

CountVectorizer(binary=True, lowercase=False,
                vocabulary=['hi', 'hey', 'anyone', 'there', 'hello', 'up',
                            'see', 'goodbye', 'am', 'leaving', 'have', 'go',
                            'rush', 'dexter', 'should', 'call', 'when', 'guys',
                            'open', 'of', 'operation', 'timing', 'id', 'like',
                            'order', 'on', 'menu', 'do', 'reccommend', 'could', ...])

In [14]:
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [16]:
vectorizer = joblib.load("vectorizer.pkl")

In [17]:
ohe = vectorizer.transform(docs_x)

In [18]:
ohe

<55x80 sparse matrix of type '<class 'numpy.int64'>'
	with 168 stored elements in Compressed Sparse Row format>

In [19]:
le = preprocessing.LabelEncoder()

In [20]:
docs_y = le.fit_transform(docs_y)

In [21]:
docs_y

array([6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 5, 5, 5, 5, 5,
       5, 5, 5, 1, 1, 1, 1, 1, 9, 9, 9, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 4,
       4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0], dtype=int64)

In [22]:
le.classes_

array(['address', 'age', 'available_tables', 'book_table', 'contact',
       'goodbye', 'greeting', 'hours', 'menu', 'name'], dtype='<U16')

In [23]:
doc_input = Input(shape=(80,))
dense_layer_1 = Dense(32,activation='relu',kernel_initializer='glorot_normal')(doc_input)
dense_layer_2 = Dense(64,activation='relu',kernel_initializer='glorot_normal')(dense_layer_1)
drop_out = Dropout(0.6)(dense_layer_2)
output=Dense(len(labels), activation='softmax')(drop_out)

In [24]:
doc_y = to_categorical(docs_y,num_classes=len(labels))

In [25]:
model = Model(doc_input,output)

In [26]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2592      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 5,354
Trainable params: 5,354
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
model.fit(ohe,doc_y,epochs=200,batch_size=400,verbose=1)

Epoch 1/200
1/1 [==============================] - 0s 4ms/step - loss: 2.3019 - accuracy: 0.0727
Epoch 2/200
1/1 [==============================] - 0s 0s/step - loss: 2.2887 - accuracy: 0.1455
Epoch 3/200
1/1 [==============================] - 0s 979us/step - loss: 2.2704 - accuracy: 0.1273
Epoch 4/200
1/1 [==============================] - 0s 998us/step - loss: 2.3156 - accuracy: 0.0545
Epoch 5/200
1/1 [==============================] - 0s 999us/step - loss: 2.2854 - accuracy: 0.1273
Epoch 6/200
1/1 [==============================] - 0s 0s/step - loss: 2.2874 - accuracy: 0.1455
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 2.2801 - accuracy: 0.1455
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 2.2310 - accuracy: 0.1636
Epoch 9/200
1/1 [==============================] - 0s 996us/step - loss: 2.2662 - accuracy: 0.1273
Epoch 10/200
1/1 [==============================] - 0s 2ms/step - loss: 2.2293 - accuracy: 0.1636
Epoch 11/200
1/1 [=====

1/1 [==============================] - 0s 998us/step - loss: 1.6671 - accuracy: 0.6364
Epoch 84/200
1/1 [==============================] - 0s 998us/step - loss: 1.5699 - accuracy: 0.6909
Epoch 85/200
1/1 [==============================] - 0s 998us/step - loss: 1.6662 - accuracy: 0.5455
Epoch 86/200
1/1 [==============================] - 0s 998us/step - loss: 1.5763 - accuracy: 0.6364
Epoch 87/200
1/1 [==============================] - 0s 995us/step - loss: 1.5936 - accuracy: 0.6364
Epoch 88/200
1/1 [==============================] - 0s 998us/step - loss: 1.5503 - accuracy: 0.6727
Epoch 89/200
1/1 [==============================] - 0s 996us/step - loss: 1.4797 - accuracy: 0.7091
Epoch 90/200
1/1 [==============================] - 0s 998us/step - loss: 1.5525 - accuracy: 0.6545
Epoch 91/200
1/1 [==============================] - 0s 994us/step - loss: 1.4920 - accuracy: 0.7091
Epoch 92/200
1/1 [==============================] - 0s 998us/step - loss: 1.4954 - accuracy: 0.6909
Epoch 93/200


1/1 [==============================] - 0s 997us/step - loss: 0.5109 - accuracy: 0.9455
Epoch 165/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4637 - accuracy: 0.9818
Epoch 166/200
1/1 [==============================] - 0s 998us/step - loss: 0.5596 - accuracy: 0.9273
Epoch 167/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4708 - accuracy: 0.9818
Epoch 168/200
1/1 [==============================] - 0s 998us/step - loss: 0.4128 - accuracy: 0.9636
Epoch 169/200
1/1 [==============================] - 0s 2ms/step - loss: 0.5370 - accuracy: 0.8909
Epoch 170/200
1/1 [==============================] - 0s 997us/step - loss: 0.5358 - accuracy: 0.9091
Epoch 171/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4990 - accuracy: 0.9273
Epoch 172/200
1/1 [==============================] - 0s 998us/step - loss: 0.4555 - accuracy: 0.9273
Epoch 173/200
1/1 [==============================] - 0s 997us/step - loss: 0.4522 - accuracy: 0.9273
Epoch 174/20

In [29]:
np.argmax(model.predict(ohe[0]),-1)

array([6], dtype=int64)

In [30]:
query = ["Can you recommend something ?"]

In [31]:
np.argmax(model.predict(vectorizer.transform(query)))

8

In [32]:
le.classes_

array(['address', 'age', 'available_tables', 'book_table', 'contact',
       'goodbye', 'greeting', 'hours', 'menu', 'name'], dtype='<U16')

In [33]:
joblib.dump(model,"final_model.pkl")

TypeError: cannot pickle '_thread.RLock' object

In [24]:
model.save("final_model.h5")

In [ ]:
def get_input(query):
    inp = vectorizer.transform([query])
    return inp